In [1]:
import sys
sys.path.append("..")
from datasets import load_from_disk, Dataset
from transformers import AutoTokenizer
import numpy as np
from tqdm import tqdm, trange
import torch
import torch.nn.functional as F
from retrieval_model import BertEncoder
from torch import nn
import pandas as pd
from torch.utils.data import (DataLoader, RandomSampler, TensorDataset, SequentialSampler)
from tqdm import tqdm, trange
import pickle

In [2]:
p_encoder = BertEncoder.from_pretrained("/opt/ml/mrc-level2-nlp-08/retrieval/p_encoder")
q_encoder = BertEncoder.from_pretrained("/opt/ml/mrc-level2-nlp-08/retrieval/q_encoder")
tokenizer =  AutoTokenizer.from_pretrained("klue/bert-base")

In [3]:
def get_relevant_doc(q_encoder, p_encoder, query, context, k=100):
    if torch.cuda.is_available():
        p_encoder.cuda()
        q_encoder.cuda()
    with torch.no_grad():
        p_encoder.eval()
        q_encoder.eval()

        q_seqs_val = tokenizer([query], padding="max_length", truncation=True, return_tensors='pt').to('cuda')
        q_emb = q_encoder(**q_seqs_val).to('cpu')  #(num_query, emb_dim)

        p_embs = []
        for p in context:
            p = tokenizer(p, padding="max_length", truncation=True, return_tensors='pt').to('cuda')
            p_emb = p_encoder(**p).to('cpu').numpy()
            p_embs.append(p_emb)

    p_embs = torch.Tensor(p_embs).squeeze()  # (num_passage, emb_dim)
    dot_prod_scores = torch.matmul(q_emb, torch.transpose(p_embs, 0, 1))

    rank = torch.argsort(dot_prod_scores, dim=1, descending=True).squeeze()

    return_context = []
    for i in range(k):
        return_context.append(context[rank[i]])
    
    return dot_prod_scores.squeeze(), return_context #,rank[:k]

In [7]:
with open('/opt/ml/data/elastic_retrieval.bin','rb') as f:
    elastic_valid = pickle.load(f)

In [8]:
querys = list(elastic_valid.keys())

In [9]:
hybrid = {}
for i in tqdm(range(len(querys))):
    _, re_rank = get_relevant_doc(q_encoder,p_encoder,querys[i],elastic_valid[querys[i]])
    hybrid[querys[i]] = re_rank


100%|██████████| 600/600 [18:51<00:00,  1.89s/it]


In [11]:
with open("/opt/ml/data/hybrid_retrieval.bin", "wb") as file:
    pickle.dump(hybrid,file)

In [27]:
idx = 14
print(querys[idx])
hybrid[querys[idx]][:20]

괴수로부터 메구밍을 구해준 사람은 누구인가?


['홍마족의 가난한 집에서 태어났으며 끼니도 제대로 먹지 못해 빈곤한 어린 시절을 보냈다. 코멧코만 할 때에 심심해서 사신을 봉인하는 조각을 퍼즐처럼 맞추다가 봉인이 풀어버렸고, 거대한 마수가 메구밍을 덮치자 그 때 한 언니가 폭렬 마법을 이용해 마수를 물리친다. 그 때 본 폭렬 마법이 기억에 남아서 그 언니에게 폭렬 마법을 배워 후에 이를 습득해 그 언니에게 보여주자는 마음을 품게 된다. 이후 홍마족 학교에 들어간 메구밍은 매우 우수한 성적을 거뒀고, 꿈에도 그리던 폭렬 마법을 마침내 습득하여 학교를 졸업하였으나, 액셀 마을에 폭렬 마법을 쓰는 여성이 있다는 정보를 듣게 된다. 그래서 잘 하지도 못하는 아르바이트를 뛰며 여행 자금을 모아 융융과 아르칸레티아를 거쳐 액셀 마을에 도착한다. 하지만 초보자 마을이라도 폭렬 마법밖에 쓰지 못하는 마법사는 어디에서나 받아주지 않았고, 오히려 소동만 벌이는 바람에 평판만 떨어졌다. 그러던 중, 융융과 헤어지고 사흘 간 자금을 탕진하고 굶고 다니다 상급직을 모집하는 파티 모집 공고를 보게 되고, 그 파티에 합류하여 진정한 동료들을 찾게 된다.',
 '다마르울란은 빠띠 로겐데르 수상의 조카로 태어난 왕자이지만 할아버지의 은둔처에서 자랐다. 할아버지의 충고에 따라서, 다마르울란은 세상적인 경험과 직업을 구하기 위해 마자파힛 법정으로 가게 되었다. 그의 사촌인 오만한 라양 세타와 수상의 아들 인 라양 쿠미티르는 다마르울란이 돌아왔을 때, 그를 학대했다. 빠띠 로겐데르는 다마르울란이 자신의 아들들과 경쟁하는 것을 원하지 않았고, 그를 잔디 깎는 사람과 마구간지기 소년으로, 로겐데르의 말을 돌보는 일을 시켰다. 비록 다마르울란은 양질의 옷을 벗고 마구간에서 보잘것 없는 하인으로 일하였지만 다마르울란은 여전히 눈에 띄도록 멋져 보였다. 잘생긴 마구간 소년에 대한 소문이 결국 수상의 딸인 안자스마라에게 전해진다. 안자스마라는 다마르울란을 몰래 찾아 내고, 서로 사랑에 빠지고, 아무도 모르게 결혼한다. 어느 날 밤, 안자스마라의 형제들